In [2]:
import pandas as pd
import numpy as np
from statsmodels.stats.multitest import fdrcorrection
from copy import deepcopy

In [5]:
import os
os.chdir("G:\האחסון שלי\סמסטר 6\גנטיקה בעידן הרפואה האישית\ex3\System-Genetics-Assignment-3")

In [26]:
def create_gene_map_loc(df, p_vals):
    df = deepcopy(df)
    df.index = df['marker symbol']
    df = df[['representative genome chromosome', 'representative genome start', 'representative genome end']]
    relevant_gene_loc = df[df.index.isin(p_vals.index)]
    relevant_gene_loc.columns = ['chromosome', 'start', 'end']
    df = df.dropna(inplace=False)
    return relevant_gene_loc, df

In [28]:
mgi = pd.read_csv("MGI_Coordinates.Build37.rpt.txt", sep='\t', header=0)
p_vals = pd.read_csv("after association test.csv", header=0)
p_vals.drop("Unnamed: 0", axis=1, inplace=True)
p_vals.index = p_vals['data']   ##### IMPORTTANT LINE!!!!
genotypes = pd.read_excel("filtered_genotypes.xls")
genotypes.drop("Unnamed: 0", axis=1, inplace=True)
locis = list(p_vals.columns)
locis.remove('data')

In [29]:
relevant, loc2 = create_gene_map_loc(mgi, p_vals)

In [18]:
def cis_trans(snp, gene):
    if int(gene['chromosome']) == snp['Chr_Build37']:
        position_snp = snp['Build37_position']
        if np.abs(position_snp - gene['start']) <= 2000000 or np.abs(position_snp - gene['end']) <= 2000000:
            return 'cis'
        else:
            return 'trans'
    return 'trans'

In [19]:
def all_loci(gene, genotypes):
    d = {}
    for i in range(len(genotypes)):
        d[i] = cis_trans(genotypes.iloc[i], gene)
    return pd.Series(d)

In [82]:
def cis_trans_for_all_loci(genes, genotypes, locis):
    d = {}
    for i in range(len(genes)):
        d[genes.iloc[i].name] = (all_loci(genes.iloc[i], genotypes))
    d = pd.DataFrame(d).T
    d.index.rename('marker symbol', inplace=True)
    d = d.rename({i: locis[i] for i in range(len(list(d.columns)))}, axis='columns')
    return d

In [83]:
cis_trans_df = cis_trans_for_all_loci(relevant, genotypes, locis)

In [84]:
cis_trans_df

,0,2,3,4,6,10,13,15,21,23,...,3783,3784,3785,3786,3787,3788,3789,3791,3794,3795
marker symbol,,,,,,,,,,,,,,,,,,,,,
Cd72,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Fos,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Ptgs2,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Cd33,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Fdft1,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Ap1m1,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Vcan,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Lta,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans
Enah,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans,...,trans,trans,trans,trans,trans,trans,trans,trans,trans,trans


In [34]:
check_ariel = pd.read_csv(r"G:\האחסון שלי\סמסטר 6\גנטיקה בעידן הרפואה האישית\אריאל\תרגיל 3\hw3_209881531\cis_trans.csv")

In [43]:
check_ariel = check_ariel.reset_index()

In [85]:
check_ariel.drop('index', axis=1).set_index('marker symbol').equals(cis_trans_df)

True

In [ ]:
# CIS-TRANS COMPLETE. COPY TO CODE 
# DO NOT FORGET TO COPY CORRECTIONS FROM FILTERING CODE!!!
# PROCEED WITH CODE THEN ANSWER QUESTIONS!!!
# TRY TO ASK FOR PROLONGING DUE DATE DUE TO EXAMS